http://jira.lge.com/issue/browse/HEDATAPLFM-1291?attachmentSortBy=dateTime&attachmentOrder=asc

In [0]:
%sql
select 
  date_ym, mac_addr, X_Device_Sales_Model,X_Device_Platform, normal_log
from aic_data_ods.tlamp.normal_log_webos25
where 1=1
  and date_ym = '2025-12'
  and X_Device_Country = 'US'
  and X_Device_Platform like '%25%'
  and context_name = 'lginput2' and message_id = 'NL_PAIRING_CHECK'

In [0]:
%sql
select 
  date_ym, mac_addr, X_Device_Sales_Model,X_Device_Platform, normal_log
from aic_data_ods.tlamp.normal_log_webos25
where 1=1
  and date_ym = '2025-12'
  and X_Device_Country = 'US'
  and X_Device_Platform like '%25%'
  and context_name = 'ICS' and message_id = 'NL_UVC_LIST'

In [0]:
%sql
create table if not exists adhoc.heds.heds_1291 as 
WITH base AS (
  SELECT
      date_ym,
      mac_addr,
      X_Device_Sales_Model,
      X_Device_Platform,
      context_name,
      message_id,
      normal_log,
      log_create_time
  FROM aic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
    AND date_ym = '2025-12'
    AND X_Device_Country = 'US'
    AND X_Device_Platform LIKE '%25%'
    AND (
      (context_name = 'ICS'      AND message_id = 'NL_UVC_LIST')
      OR
      (context_name = 'lginput2' AND message_id = 'NL_PAIRING_CHECK')
    )
),

-- 메시지별 최신 로그(= log_create_time 최댓값)만 남기기
nl_uvc_list_latest AS (
  SELECT date_ym, mac_addr, normal_log,
         ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn
  FROM base
  WHERE context_name = 'ICS' AND message_id = 'NL_UVC_LIST'
),
nl_pairing_check_latest AS (
  SELECT date_ym, mac_addr, normal_log,
         ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn
  FROM base
  WHERE context_name = 'lginput2' AND message_id = 'NL_PAIRING_CHECK'
),

-- DIMENSION 키(date_ym, mac_addr)
dim_keys AS (
  SELECT DISTINCT date_ym, mac_addr FROM base
),

-- 단말 속성(모델/플랫폼)은 두 메시지를 통틀어 가장 최신 log_create_time 1건에서 취함
latest_device_attr AS (
  SELECT date_ym, mac_addr, X_Device_Sales_Model, X_Device_Platform
  FROM (
    SELECT
      date_ym, mac_addr, X_Device_Sales_Model, X_Device_Platform, log_create_time,
      ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn
    FROM base
  ) t
  WHERE rn = 1
)

SELECT
  d.date_ym,
  d.mac_addr,
  a.X_Device_Sales_Model,
  a.X_Device_Platform,
  u.normal_log AS normal_log_of_NL_UVC_LIST,
  p.normal_log AS normal_log_of_NL_PAIRING_CHECK
FROM dim_keys d
LEFT JOIN latest_device_attr a
  ON d.date_ym = a.date_ym AND d.mac_addr = a.mac_addr
LEFT JOIN (
  SELECT date_ym, mac_addr, normal_log
  FROM nl_uvc_list_latest
  WHERE rn = 1
) u
  ON d.date_ym = u.date_ym AND d.mac_addr = u.mac_addr
LEFT JOIN (
  SELECT date_ym, mac_addr, normal_log
  FROM nl_pairing_check_latest
  WHERE rn = 1
) p
  ON d.date_ym = p.date_ym AND d.mac_addr = p.mac_addr
;

In [0]:
%sql
select count(1)
from adhoc.heds.heds_1291

In [0]:
%sql
select * 
from (
  select 
    row_number() over (order by date_ym, mac_addr) as rn,
    *
  from adhoc.heds.heds_1291
)
where  rn between 0 and 800000

In [0]:
%sql
select * 
from (
  select 
    row_number() over (order by date_ym, mac_addr) as rn,
    *
  from adhoc.heds.heds_1291
)
where  rn between 800001 and 1600000